In [46]:
import pandas as pd
import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import librosa
import librosa.display
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline


In [65]:
#dividing to test, validation and training set
DATASET_AUDIO_PATH = "Data/genres_original/"

def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

# Seed to use when shuffling the dataset and the noise
SHUFFLE_SEED = 43

# Percentage of samples to use for validation and test
VALID_and_TEST_SPLIT = 0.2

# Percentage of samples to use for validation and test
VALID_SPLIT = 0.5


BATCH_SIZE = 128
EPOCHS = 100
SAMPLING_RATE = 16000

audio_paths = [] # list of paths for every audio sample
labels = [] # classification
for label, name in enumerate(class_names):
    print("Processing speaker {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names))
)


# Shuffle for equals splitting
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

# 80:10:10
# Split into training, validation and test
num_val_samples = int(VALID_and_TEST_SPLIT * len(audio_paths))
print("Using {} files for training.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Using {} files for validation and test.".format(num_val_samples))
valid_test_audio_paths = audio_paths[-num_val_samples:]
valid_test_labels = labels[-num_val_samples:]

num_test_samples = int(VALID_SPLIT * len(valid_test_audio_paths))
print("Using {} files for test.".format(num_test_samples))
test_audio_paths = audio_paths[-num_test_samples:]
test_labels = labels[-num_test_samples:]

print("Using {} files for validation.".format(num_test_samples))
valid_audio_paths = audio_paths[:-num_test_samples]
valid_labels = labels[:-num_test_samples]


# Create 3 datasets, one for training, one for validation and the last for test
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)
valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)

test_ds = paths_and_labels_to_dataset(test_audio_paths, test_labels)
test_ds = test_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)

####
def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)


valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)


test_ds = test_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)



Our class names: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
Processing speaker blues
Processing speaker classical
Processing speaker country
Processing speaker disco
Processing speaker hiphop
Processing speaker jazz
Processing speaker metal
Processing speaker pop
Processing speaker reggae
Processing speaker rock
Found 1000 files belonging to 10 classes.
Using 800 files for training.
Using 200 files for validation and test.
Using 100 files for test.
Using 100 files for validation.
Cause: could not parse the source code:

    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lamb

In [66]:
# model
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))

In [67]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 8000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 8000, 16)     64          input[0][0]                      
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 8000, 16)     0           conv1d_19[0][0]                  
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 8000, 16)     784         activation_13[0][0]              
____________________________________________________________________________________________

In [68]:

# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [69]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Train for 7 steps, validate for 29 steps
Epoch 1/100
7/7 [==============================] - 69s 10s/step - loss: 3.7585 - accuracy: 0.1187 - val_loss: 2.2527 - val_accuracy: 0.1356
Epoch 2/100
7/7 [==============================] - 60s 9s/step - loss: 2.1716 - accuracy: 0.2125 - val_loss: 2.0019 - val_accuracy: 0.2456
Epoch 3/100
7/7 [==============================] - 58s 8s/step - loss: 1.8748 - accuracy: 0.3100 - val_loss: 1.7561 - val_accuracy: 0.3656
Epoch 4/100
7/7 [==============================] - 60s 9s/step - loss: 1.6835 - accuracy: 0.4087 - val_loss: 1.6702 - val_accuracy: 0.4411
Epoch 5/100
7/7 [==============================] - 63s 9s/step - loss: 1.6013 - accuracy: 0.4563 - val_loss: 1.4493 - val_accuracy: 0.4944
Epoch 6/100
7/7 [==============================] - 62s 9s/step - loss: 1.4484 - accuracy: 0.5038 - val_loss: 1.4373 - val_accuracy: 0.5222
Epoch 7/100
7/7 [==============================] - 70s 10s/step - loss: 1.3913 - accuracy: 0.5300 - val_loss: 1.3651 - val_a